In [ ]:
!git clone https://github.com/andreazenotto/mesotheliomaSemiSupervisedClassification.git

In [ ]:
import os
from tqdm import tqdm
import sys
import shutil
import numpy as np
import tensorflow as tf

sys.path.append('mesotheliomaSemiSupervisedClassification/src')

from simclr import *
from attention_mil import *

## Inference
We evaluate the performances of our proposal model on the <em>**camel-inference-dataset**</em>

This dataset is made of 8 WSI: 4 epithelioid and 4 biphasic, used for testing purpouse

In [3]:
# Evaluation Dataset
test_patches_dir = "/kaggle/input/camel-inference-dataset"

In [ ]:
backbone_model_dir = '/kaggle/input/camel-backbone-model/tensorflow2/default/1/best_backbone.h5'
backbone = tf.keras.models.load_model(backbone_model_dir)

In [6]:
featuresTest, labelsTest = extract_features(test_patches_dir, backbone, batch_size=256)
datasetTest = generate_dataset(featuresTest, labelsTest, num_classes=3, batch_size=1)

Extracting features: 100%|██████████| 8/8 [04:07<00:00, 30.90s/it]


In [7]:
attention_model_dir = "/kaggle/input/camel-mil-model/keras/default/1/best_mil.keras"
model = tf.keras.models.load_model(attention_model_dir, custom_objects = {'MultiHeadAttentionMIL': MultiHeadAttentionMIL})

In [14]:
wrong = {
    'epithelioid': 0,
    'sarcomatoid': 0,
    'biphasic': 0
}

def hashMapPred(x):
    x.astype(np.int64)
    labels = ['epithelioid', 'sarcomatoid', 'biphasic']
    return labels[x]

correct = 0
total = 0

for x, y in datasetTest:
    pred = mhaMIL.predict(x, verbose=0)[0]
    # label corrente: np.argmax(y[0]) return the index 0 in case of 1, 0, 0; the index 1 in case of 0, 1, 0 and so on
    label_y = hashMapPred(np.argmax(y[0]))

    prediction = hashMapPred(np.argmax(pred))
        
    if prediction != label_y:
        wrong[label_y] += 1
    if prediction == label_y:
        correct += 1
        print(f"✅ Prediction = {prediction}\t label = {label_y}\t Probabilities = {pred}")
    else:
        print(f"❌ Prediction = {prediction}\t label = {label_y}\t Probabilities = {pred}")
    total += 1

print(f"\n-------------------\nTotal accuracy: {correct/total:.2%}\n-------------------\n")
print(f"Total miss-classifications per subtype = {wrong}")

✅ Prediction = epithelioid	 label = epithelioid	 Probabilities = [0.38952363 0.3545914  0.25588495]
✅ Prediction = biphasic	 label = biphasic	 Probabilities = [0.3628385  0.2351986  0.40196294]
✅ Prediction = epithelioid	 label = epithelioid	 Probabilities = [0.50413245 0.19471645 0.30115104]
✅ Prediction = biphasic	 label = biphasic	 Probabilities = [0.26508182 0.3542476  0.38067052]
❌ Prediction = sarcomatoid	 label = epithelioid	 Probabilities = [0.28269762 0.36342648 0.3538759 ]
✅ Prediction = epithelioid	 label = epithelioid	 Probabilities = [0.5352522  0.1802229  0.28452483]
✅ Prediction = biphasic	 label = biphasic	 Probabilities = [0.33315933 0.25037387 0.41646683]
✅ Prediction = biphasic	 label = biphasic	 Probabilities = [0.42026716 0.15324026 0.4264926 ]

----------------------
Total accuracy: 87.50%
----------------------

Total miss-classifications per subtype = {'epithelioid': 1, 'sarcomatoid': 0, 'biphasic': 0}
